Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Aleksandrov Vladislav BPM182"
COLLABORATORS = "-"

---

In [2]:
import numpy as np
from numpy.testing import assert_allclose

# 1. Степенная итерация

Дана квадратная $n \times n$ матрица $A$. Алгоритм степенной итерации позволяет найти максимальное собственное значение $\lambda$ матрицы $A$. Для начала необходимо задать начальное приближение собственного вектора $x_0$. 


Модифицированный алгоритм степенной итерации:

1. $y_{k+1} = A x_k$

2. $x_{k+1} = \cfrac{y_{k+1}}{||y_{k+1}||}$

3. $\lambda^{(k)} = (x_k, y_{k+1})$

где $k$ - номер итерации.

Напишите функцию, реализующую поиск максимального собственного значения заданной матрицы с помощью алгоритма степенной итерации.

In [3]:
def power_iter(a, vec, eps=1e-3, maxiter=1000):
    """Реализует алгоритм степенной итерации поиска максимального собственного значения матрицы `a` 
        с начальным приближением собственного вектора vec.
    
    Parameters
    ----------
    a : array-like of floats, shape (n, n)
        Введённая матрица
    vec : array-like of floats, shape (n,)
        Введённое начальное приближение собственного вектора
    eps : float
        Введённая абсолютная точность
    maxiter : integer
        Максимальное количество итераций
    
    Returns
    -------
    lm : float
        Приближение максимального собственного значения
    outvec : array of floats, shape (n,)
        Приближение собственного вектора
    iters : integer
        Количество итераций
    """
    a = np.asarray(a, dtype=float)
    if a.ndim != 2:
        raise ValueError("a.ndim = %s, expected 2" % a.ndim)
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if (a.shape[0] != a.shape[1]):
        raise ValueError("a is not square")
        
    if (a.shape[0] != vec.shape[0]):
        raise ValueError("vec should be of length %s" % a.shape[0])
    
    x = vec
    for k in range(maxiter):
      y_next = a @ x
      x_next = y_next / np.linalg.norm(y_next)
      lym = x @ y_next
      if max(abs(y_next - lym * x)) < eps or k == maxiter - 1:
        return lym, x, k+1
      x = x_next

Протестируйте свою функцию на следующих примерах:

In [4]:
A = np.asarray([[4, 34],[7, 1]])

np.random.seed(123)
x = np.random.random(size=2)

l, e, k = power_iter(A, x)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = power_iter(A, x, 1e-5)
assert_allclose(A @ e, l * e, atol=1e-5)

###BEGIN HIDDEN TESTS

A = np.asarray([[1, 2], [0, 1]])
x = np.asarray([1,1])

l, e, k = power_iter(A, x)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = power_iter(A, x, 1e-5)
assert_allclose(A @ e, l * e, atol=1e-5)

###END HIDDEN TESTS

# 2. Обратная итерация

Этот метод может использоваться, например, для поиска собственных векторов матрицы, если уже имеется достаточно хорошее приближение её максимального собственного значения.

Для реализации алгоритма необходимо задать начальные приближения для собственного вектора $x_0$ и собственного значения $\mu$.

На каждой итерации метода решается система уравнений:
$$(A - \lambda^{(k)} I) y_{k+1} = x_k,$$
приэтом $\lambda^{(0)} = \mu$. После этого за новое приближение собственного вектора принимается нормированный вектор $y_{k+1}$:
$$x_{k+1} = \cfrac{y_{k+1}}{||y_{k+1}||}.$$

In [5]:
def inverse_iter(a, vec, mu, eps=1e-3, maxiter=1000):
    """Реализует алгоритм степенной итерации поиска максимального собственного значения матрицы a 
        с начальным приближением собственного вектора vec и начальным приближением собственного значения mu
    
    Parameters
    ----------
    a : array-like of floats, shape (n, n)
        Введённая матрица
    vec : array-like of floats, shape (n,)
        Введённое начальное приближение собственного вектора
    mu : float
        Введённое начальное приближение собственного значения
    eps : float
        Введённая точность 
    maxiter : integer
        Максимальное количество итераций
    
    Returns
    -------
    lm : float
        Приближение максимального собственного значения
    outvec : array of floats, shape (n,)
        Приближение собственного вектора
    iters : integer
        Количество итераций
    """
    a = np.asarray(a, dtype=float)
    if a.ndim != 2:
        raise ValueError("a.ndim = %s, expected 2" % a.ndim)
    
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
        
    if (a.shape[0] != a.shape[1]):
        raise ValueError("a is not square")
        
    if (a.shape[0] != vec.shape[0]):
        raise ValueError("vec should be of length %s" % a.shape[0])
    
    if (np.linalg.matrix_rank(a - mu * np.eye(a.shape[0])) != a.shape[0]):
      lym = mu - 1e-3
    else:
      lym = mu
    x = vec
    for k in range(maxiter):
      new_matrix = a - lym * np.eye(a.shape[0])
      if (np.linalg.matrix_rank(new_matrix) != a.shape[0]):
        lym, x, k
      y = np.linalg.inv(new_matrix) @ x
      if max(abs(a @ x - lym * x)) < eps or k == maxiter - 1:
        return lym, x, k+1
      x = y / np.linalg.norm(y)
      lym = x @ (a @ x)

Протестируйте свою функцию на следующих примерах:

In [6]:
A = np.asarray([[4, 34],
                [7, 1]])
x = np.random.random(size=2)

l, e, k = inverse_iter(A, x, 1.)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = inverse_iter(A, x, 100.)
assert_allclose(A @ e, l * e, atol=1e-3)

###BEGIN HIDDEN TESTS

A = np.asarray([[1, 2], [0, 1]])
x = np.asarray([1, 1])

l, e, k = inverse_iter(A, x, 0.5)
assert_allclose(A @ e, l * e, atol=1e-3)

l, e, k = inverse_iter(A, x, 50.)
assert_allclose(A @ e, l * e, atol=1e-3)

###END HIDDEN TESTS

## Пример

Рассмотрите матрицу 
$$ A = 
\begin{bmatrix}
3 & 1 & 0 & 0\\ 
1 & 2 & 0 & 1\\ 
0 & 0 & 1 & 1\\ 
0 & 1 & 1 & 1
\end{bmatrix}.$$

In [7]:
# Определите матрицу `А` и задайте начальное приближение для собственного вектора
# (используйте случайный начальный вектор)

def A_and_x(seed=1234):
    """Возвращает матрицу `А` и случайный начальный вектор."""
    a = np.array([[3, 1, 0, 0], [1,2,0,1], [0,0,1,1], [0,1,1,1]])
    np.random.seed(seed)
    x = np.random.random(size=4)
    return a, x

Найдите максимальное собственное число этой матрицы тремя разными методами.

### 1) Степенная итерация

In [8]:
A, x = A_and_x()

l, e, k = power_iter(A, x, eps=1e-5)
assert_allclose(A @ e, l * e, atol=1.6e-2)

### BEGIN HIDDEN TESTS
A, x = A_and_x()
l, e, k = power_iter(A, x, eps=1e-5)
assert_allclose(A @ e, l * e, atol=1.6e-3)
### END HIDDEN TESTS

### 2) Обратная итерация с $\mu = 3.5$

In [9]:
A, x = A_and_x()

l, e, k = inverse_iter(A, x, 3.5, eps=1e-5)
assert_allclose(A @ e, l * e, atol=1e-3)

### BEGIN HIDDEN TESTS
A, x = A_and_x()
l, e, k = inverse_iter(A, x, 3.5, eps=1e-5)
assert_allclose(A @ e, l * e, atol=1e-5)
### END HIDDEN TESTS

### 3) Обратная итерация с $\mu = 3.7$

In [10]:
A, x = A_and_x()

l, e, k = inverse_iter(A, x, 3.7, eps=1e-5)
assert_allclose(A @ e, l * e, atol=1e-3)

### BEGIN HIDDEN TESTS
A, x = A_and_x()
l, e, k = inverse_iter(A, x, 3.7, 1e-5)
assert_allclose(A @ e, l * e, atol=1e-5)
### END HIDDEN TESTS

Сколько шагов $k$ требуется в каждом случае для того, чтобы получить настоящий собственный вектор $x^*$ с точностью $||x^* - x_k||_2 < 10^{-3}$?

In [11]:
A, x = A_and_x()
eg = np.linalg.eig(A)
mv = eg[1][:, eg[0].argmax()]
v1, k1 = power_iter(A, x)[1:]
v2, k2 = inverse_iter(A, x, 3.5)[1:]
v3, k3 = inverse_iter(A, x, 3.7)[1:]
print("Степенная итерация:")
print("k =", k1, "pr =", np.linalg.norm(mv + v1))
print("Уменьшим кол-во итераций:")
print("k =", power_iter(A, x, maxiter=k1 -1)[2], "pr =", np.linalg.norm(mv + power_iter(A, x, maxiter=k1 -1)[1]))

print("\nОбратная итерация с m = 3.5:")
print("k =", k2, "pr =", np.linalg.norm(mv + v2))
print("Уменьшим кол-во итераций:")
print("k =", inverse_iter(A, x, 3.5, maxiter= k2 - 1)[2], "pr =", np.linalg.norm(mv + inverse_iter(A, x, 3.5, maxiter= k2 - 1)[1]))

print("\nОбратная итерация с m = 3.7:")
print("k =", k3, "pr =", np.linalg.norm(mv + v3))
print("Уменьшим кол-во итераций:")
print("k =", inverse_iter(A, x, 3.7, maxiter= k3 - 1)[2], "pr =", np.linalg.norm(mv + inverse_iter(A, x, 3.7, maxiter= k3 - 1)[1]))

Степенная итерация:
k = 16 pr = 0.0008132159514509945
Уменьшим кол-во итераций:
k = 15 pr = 0.0013156194035643105

Обратная итерация с m = 3.5:
k = 4 pr = 2.25178021559467e-06
Уменьшим кол-во итераций:
k = 3 pr = 0.01310356945972209

Обратная итерация с m = 3.7:
k = 3 pr = 6.824221502940697e-05
Уменьшим кол-во итераций:
k = 2 pr = 0.04077824396085803
